In [1]:
import json
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score

# Load JSON file
with open('/data1/dxw_data/llm/MKT_data_mining/Multimodal/data/captions_labeled_with_time.json', 'r') as f:
    data = json.load(f)

# Load pretrained ResNet
resnet = resnet50(pretrained=False)
resnet.load_state_dict(torch.load('/data1/dxw_data/llm/resnet/resnet50-19c8e357.pth'))
resnet = nn.Sequential(*list(resnet.children())[:-1])  # Remove the classification layer
resnet.eval()

word2vec_path = '/data1/dxw_data/llm/word2vec/GoogleNews-vectors-negative300.bin.gz'
word2vec_model  = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)


/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
# Preprocess transforms for images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to apply mask, save the merged image, and extract image features
def extract_imagemask_features(image_path, mask_path, cover_path):
    image = Image.open(image_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')

    # Apply mask to the image
    image_np = np.array(image)
    mask_np = np.array(mask)
    masked_image_np = np.multiply(image_np, mask_np[:, :, None] / 255.0)
    masked_image = Image.fromarray(masked_image_np.astype(np.uint8))

    # Save the masked image
    masked_image_save_path = os.path.join(cover_path, os.path.basename(image_path))
    masked_image.save(masked_image_save_path)

    # Apply transformations
    masked_image = transform(masked_image).unsqueeze(0)

    # Extract features using ResNet
    with torch.no_grad():
        features = resnet(masked_image).squeeze().numpy()
    return features

# Ensure the cover directory exists
cover_path = '/data1/dxw_data/llm/redbook/cover'
os.makedirs(cover_path, exist_ok=True)

# Function to extract image features
def extract_image_features(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        features = resnet(image).squeeze().numpy()
    return features

# Function to extract text features
def extract_text_features(caption):
    words = caption.split()
    word_vectors = []
    for word in words:
        if word in word2vec_model:
            vector = word2vec_model[word]
            word_vectors.append(vector)
    if not word_vectors:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)

# Prepare dataset
image_features = []
text_features = []
mask_features= []
labels = []
times = []

for item in data:
    image_path = os.path.join('/data1/dxw_data/llm/redbook/data', item['image'])
    mask_path = os.path.join('/data1/dxw_data/llm/redbook/processed', item['image'])
    if os.path.exists(image_path) and os.path.exists(mask_path):
        mask_feat = extract_imagemask_features(image_path, mask_path, cover_path)
        img_feat = extract_image_features(image_path)
        txt_feat = extract_text_features(item['caption'])
        mask_features.append(mask_feat)
        image_features.append(img_feat)
        text_features.append(txt_feat)
        labels.append(item['label'])
        times.append(item['time'])


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert to numpy arrays
image_features = np.array(image_features)
mask_features = np.array(mask_features)
text_features = np.array(text_features)
labels = np.array(labels)
times = np.array(times)

In [ ]:
# ---------------------------text和mask和image同时--------------------------- #

In [23]:
# Combine image and text features
combined_features = np.hstack((mask_features, image_features))
combined_embeddings  = np.hstack((combined_features, text_features))

# Sequence length for time series prediction
sequence_length = 2

# Create sequences for time series prediction
def create_sequences(data, labels, times, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        # Ensure that we have at least `seq_length` consecutive time steps
        if times[i+seq_length-1] - times[i] < seq_length:
            x = data[i:i+seq_length]
            y = labels[i+seq_length]
            xs.append(x)
            ys.append(y)
    return np.array(xs), np.array(ys)

X, y = create_sequences(combined_embeddings, labels, times, sequence_length)

print("X.shape:", X.shape)  # Expected to be non-zero
print("y.shape:", y.shape)  # Expected to be non-zero

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
input_size = combined_embeddings.shape[1]
hidden_size = 128
output_size = 1
num_layers = 2
num_epochs = 20
learning_rate = 0.001

# Model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



X.shape: (230, 2, 4396)
y.shape: (230,)


In [24]:
from sklearn.metrics import mean_squared_error, accuracy_score
# Train the model
for epoch in range(num_epochs):
    model.train()
    outputs = model(torch.tensor(X_train, dtype=torch.float32).to(device))
    optimizer.zero_grad()
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).to(device))
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predicted = model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().numpy()
    test_loss = mean_squared_error(y_test, predicted)
    print(f'Test Loss: {test_loss:.4f}')
    
    # If the task is classification, calculate accuracy
    if output_size == 1:  # Adjust the condition based on your task
        predicted_labels = (predicted > 0.5).astype(int)
        y_test_labels = (y_test > 0.5).astype(int)
        accuracy = accuracy_score(y_test_labels, predicted_labels)
        print(f'Test Accuracy: {accuracy:.4f}')

Epoch [1/20], Loss: 0.4117
Epoch [2/20], Loss: 0.2511
Epoch [3/20], Loss: 0.2815
Epoch [4/20], Loss: 0.2648
Epoch [5/20], Loss: 0.2466
Epoch [6/20], Loss: 0.2457
Epoch [7/20], Loss: 0.2522
Epoch [8/20], Loss: 0.2554
Epoch [9/20], Loss: 0.2536
Epoch [10/20], Loss: 0.2495
Epoch [11/20], Loss: 0.2459
Epoch [12/20], Loss: 0.2443
Epoch [13/20], Loss: 0.2448
Epoch [14/20], Loss: 0.2463
Epoch [15/20], Loss: 0.2475
Epoch [16/20], Loss: 0.2475
Epoch [17/20], Loss: 0.2466
Epoch [18/20], Loss: 0.2453
Epoch [19/20], Loss: 0.2445
Epoch [20/20], Loss: 0.2442
Test Loss: 0.2506
Test Accuracy: 0.5435


/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([184])) that is different to the input size (torch.Size([184, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
# ---------------------------mask和image同时--------------------------- #

In [ ]:
# ---------------------------mask和text同时--------------------------- #

In [ ]:
# ---------------------------image和text同时--------------------------- #

In [ ]:
# ---------------------------只用mask--------------------------- #

In [4]:
# ---------------------------只用image--------------------------- #

In [6]:
# ---------------------------只用text--------------------------- #

# 其实也等价于多模态LLM模型合并图片文本--->文本